Rand Agent Table with RNN for Time Series: for ONE AGENT ONLY

This is a test of concept notebook for predicting the next agent row for a given track_id in 1 scene.

In [ ]:
import keras
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
folderpath = "*"
file = folderpath + "\\lyftlong\\rand_agents_table0.csv"
at = pd.read_csv(file)
file = folderpath + "\\lyftlong\\rand_frames_table1.csv"
ft = pd.read_csv(file)

In [ ]:
at = at.merge(ft[['frame_index', 'scene_index']], on='frame_index')

In [ ]:
atcols = ['centroid_x', 'centroid_y', 'extent_x', 'extent_y', 'extent_z',
          'velocity_x', 'velocity_y', 'yaw', 'track_id','frame_index', 'scene_index']

In [ ]:
atrnn = at[atcols]

In [ ]:
atrnn1 = atrnn[atrnn.track_id == 1]

In [ ]:
atrnn1

In [ ]:
atrnn1 = atrnn1[atrnn1.scene_index == 7827]

In [ ]:
atrnn1.columns

In [ ]:
atrnncols = ['centroid_x', 'centroid_y', 'extent_x', 'extent_y', 'extent_z',
                  'velocity_x', 'velocity_y', 'yaw']

In [ ]:
atrnn1 = atrnn1[atrnncols]

In [ ]:
# atrnn1

In [ ]:
def make_partitions(ar: np.array, step: int = 3) -> np.array:
    len_ar = len(ar)
    partitions = np.array([])
    for i in range(len_ar - step):
        X_partition = ar[i: i + step]
        y_partition = ar[i + step]
        partition = dict()
        partition['X'] = ((X_partition))
        partition['y'] = ((y_partition))
        partitions = np.append(partitions, partition)
    return partitions

In [ ]:
partitions = make_partitions(np.array(atrnn1), 3)

In [ ]:
TRAIN_PROPORTION = 0.8
TRAIN_PROPORTION_INDEXER = int(len(atrnn1) * 0.8)

In [ ]:
X_train = []
y_train = []
X_test = []
y_test = []
for i in range(len(partitions)): # [:TRAIN_PROPORTION_INDEXER]:
    partition = partitions[i]
    if i <= TRAIN_PROPORTION_INDEXER:
        X_train.append(partition['X'])
        y_train.append(partition['y'])
    else:
        X_test.append(partition['X'])
        y_test.append(partition['y'])        
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
n_steps = 3
n_features = 8

In [ ]:
# X_train = np.reshape(X_train, (X_train.shape[0], 3, n_features))

In [ ]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(X_train, y_train, epochs=300, verbose=0)

In [ ]:
x_inp = X_test[0]
y_targ = y_test[0]

In [ ]:
x_inp = x_inp = x_inp.reshape((1, n_steps, n_features))
x_inp.shape

In [ ]:
y_hat = model.predict(x_inp, verbose=0)

In [ ]:
y_hat

In [ ]:
y_targ

In [ ]:
np.sqrt(((y_hat - y_targ)**2)).sum()

In [ ]:
from scipy import spatial

result = 1 - spatial.distance.cosine(y_hat, y_targ)
result